# Movies recommandation model

## 1. Imports

### 1.1 Libraries

In [17]:
# builtin
import os, time, sys, random
from dotenv import load_dotenv
from pathlib import Path

# data
import pandas as pd
import numpy as np

# ML
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler, normalize

# other
import warnings
warnings.filterwarnings("ignore")
import joblib
import pickle

In [18]:
BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / 'data'

print(f"Répertoire de sauvegarde: {DATA_DIR}\n")

Répertoire de sauvegarde: C:\Users\Melvin\Desktop\DATA\PORTFOLIO\Recommandation de films\data



### 1.2 Loading data

In [19]:
df = pd.read_csv(BASE_DIR / 'data' / 'df_movies_preprocess.csv')
genre_df = pd.read_csv(BASE_DIR / 'data' / 'genres_binarized.csv')

In [20]:
liste_titres = df['Titre'].unique().tolist()

In [21]:
liste_titres

['Greenland : Migration',
 'La Femme de ménage',
 'Team Démolition',
 'Zootopie 2',
 "The Shadow's Edge",
 'Oscar Shaw',
 "La Reine du crime présente : Meurtre à l'Ambassade",
 'Avatar : De feu et de cendres',
 'Predator : Badlands',
 '96 Minutes',
 'Anaconda',
 'The Internship',
 'Hamnet',
 'La Guerre des mondes',
 'The Rip',
 'David',
 'The Rule of Jenny Pen',
 'Shelter',
 'Dust Bunny',
 'Send Help',
 'The Confession',
 'Demon Slayer : Kimetsu no Yaiba - Le film : La Forteresse infinie',
 "Bob l'éponge, le film : Un pour tous, tous pirates !",
 'Icefall',
 'The Muppet Show',
 'Even If This Love Disappears Tonight',
 'The Plague',
 'xXx',
 'Insaisissables 3',
 'Iron Lung',
 'Strangers',
 'The Tank',
 'Relationship Goals',
 'Avengers',
 'KPop Demon Hunters',
 'Trap House',
 'Silent Night, Deadly Night',
 'Sinners',
 'Dhurandhar',
 'Killer Whale',
 'Dracula',
 'Marty Supreme',
 'Exit 8',
 'We Bury the Dead',
 "Une bataille après l'autre",
 'Jurassic World : Renaissance',
 'Interstellar'

## 1. SYNOPSIS : Embeddings

In [23]:
embeddings_path = DATA_DIR / 'synopsis_embeddings.npy'

if embeddings_path.exists():
    synopsis_embeddings = np.load(embeddings_path)
else:
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    synopsis_embeddings = model.encode(
        df['clean_synopsis_str'].fillna('').tolist(),
        show_progress_bar=True
    )
    np.save(embeddings_path, synopsis_embeddings)
    print("Embeddings saved")

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 847.13it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 187/187 [01:02<00:00,  2.97it/s]

Embeddings saved


In [ ]:
# Pondération
synopsis_embeddings_weighted = synopsis_embeddings * 3.0

## 2. TITRE : TF-IDF

In [25]:
titre_tfidf = TfidfVectorizer(max_features=500).fit_transform(df['Titre'])
titre_tfidf_weighted = titre_tfidf * 1.0

## 3. FEATURES NUMÉRIQUES

In [50]:
note_scaled = MinMaxScaler().fit_transform(df[['Note']]) * 1
age_scaled = MinMaxScaler().fit_transform(df[['Age du film']]) * 0.75
pop_scaled = MinMaxScaler().fit_transform(df[['Popularité']]) * 0.5

   - Note normalisée: (5958, 1)
   - Âge normalisé: (5958, 1)
   - Popularité normalisée: (5958, 1)


## 4. GENRES

In [27]:
genre_columns = list(genre_df.columns)
genre_matrix = df[genre_columns].values * 1.5

## 5. COMBINER (méthode dense)

In [29]:

features_vectorized = np.hstack([
    titre_tfidf_weighted.toarray(),
    synopsis_embeddings_weighted,
    note_scaled,
    age_scaled,
    pop_scaled,
    genre_matrix
])

print(f"✅ Features combinées: {features_vectorized.shape}")
print(f"   Dimensions par feature:")
print(f"   • Titre (TF-IDF):    {titre_tfidf_weighted.shape[1]}")
print(f"   • Synopsis (Embed):  {synopsis_embeddings_weighted.shape[1]}")
print(f"   • Note:              1")
print(f"   • Âge:               1")
print(f"   • Popularité:        1")
print(f"   • Genres:            {genre_matrix.shape[1]}")
print(f"   • TOTAL:             {features_vectorized.shape[1]}")

✅ Features combinées: (5958, 906)
   Dimensions par feature:
   • Titre (TF-IDF):    500
   • Synopsis (Embed):  384
   • Note:              1
   • Âge:               1
   • Popularité:        1
   • Genres:            19
   • TOTAL:             906


In [31]:
# Normalisation L2
features_vectorized = normalize(features_vectorized, norm='l2')

## 6. SIMILARITÉ COSINUS

In [32]:
print("Calcul de la similarité cosinus...")
cosine_sim = cosine_similarity(features_vectorized)

print(f"Matrice de similarité: {cosine_sim.shape}")

Calcul de la similarité cosinus...
Matrice de similarité: (5958, 5958)


In [33]:
np.save(DATA_DIR / 'cosine_sim.npy', cosine_sim)

## 3. Recommandations function

In [51]:
def get_recommendations(title, cosine_sim=cosine_sim, df=df, top_n=9):
    """
    Retourne les films les plus similaires à un film donné.
    
    Args:
        title (str): Titre du film
        cosine_sim (np.array): Matrice de similarité
        df (pd.DataFrame): DataFrame contenant les films
        top_n (int): Nombre de recommandations à retourner
    
    Returns:
        pd.DataFrame: DataFrame avec les colonnes ['Titre', 'Affiche']
    """
    # Trouve l'index du film
    idx = df.index[df['Titre'] == title].tolist()[0]
    
    # Récupère les scores de similarité
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Trie par score décroissant
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Prend les top_n films (en excluant le film lui-même)
    sim_scores = sim_scores[1:top_n+1]
    
    # Récupère les indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Retourne les recommandations
    recommendations = df[['Titre', 'Affiche']].iloc[movie_indices]
    
    return recommendations


In [53]:
# Test avec plusieurs films
test_films = ["Inception", "Avatar", "Titanic"]

for test_film in test_films:
    print(f"\n{'='*60}")
    print(f"Test avec le film: {test_film}")
    print(f"{'='*60}")
    
    try:
        recommendations = get_recommendations(test_film, top_n=5)
        
        print("\nTop 5 recommandations:")
        for i, (idx, row) in enumerate(recommendations.iterrows(), 1):
            print(f"  {i}. {row['Titre']}")
    except:
        print(f"Film non trouvé: {test_film}")

print("\nTests réussis!")


Test avec le film: Inception

Top 5 recommandations:
  1. Rogue One : A Star Wars Story
  2. Iron Man 3
  3. Ant-Man et la Guêpe
  4. Jurassic World : Renaissance
  5. Spider-Man 2

Test avec le film: Avatar

Top 5 recommandations:
  1. Avatar : La Voie de l'eau
  2. Avatar : De feu et de cendres
  3. Alienoid : Les Protecteurs du Futur
  4. X-Men : Apocalypse
  5. Pacific Rim : Uprising

Test avec le film: Titanic

Top 5 recommandations:
  1. Juliet & Romeo
  2. Rendez-vous avec le destin
  3. Une seconde chance
  4. Loveable
  5. À deux mètres de toi

Tests réussis!


# 4. Saving

In [42]:
liste_titres = df['Titre'].tolist()

In [43]:
liste_titres

['Greenland : Migration',
 'La Femme de ménage',
 'Team Démolition',
 'Zootopie 2',
 "The Shadow's Edge",
 'Oscar Shaw',
 "La Reine du crime présente : Meurtre à l'Ambassade",
 'Avatar : De feu et de cendres',
 'Predator : Badlands',
 '96 Minutes',
 'Anaconda',
 'The Internship',
 'Hamnet',
 'La Guerre des mondes',
 'The Rip',
 'David',
 'The Rule of Jenny Pen',
 'Shelter',
 'Dust Bunny',
 'Send Help',
 'The Confession',
 'Demon Slayer : Kimetsu no Yaiba - Le film : La Forteresse infinie',
 "Bob l'éponge, le film : Un pour tous, tous pirates !",
 'Icefall',
 'The Muppet Show',
 'Even If This Love Disappears Tonight',
 'The Plague',
 'xXx',
 'Insaisissables 3',
 'Iron Lung',
 'Strangers',
 'The Tank',
 'Relationship Goals',
 'Avengers',
 'KPop Demon Hunters',
 'Trap House',
 'Silent Night, Deadly Night',
 'Sinners',
 'Dhurandhar',
 'Killer Whale',
 'Dracula',
 'Marty Supreme',
 'Exit 8',
 'We Bury the Dead',
 "Une bataille après l'autre",
 'Jurassic World : Renaissance',
 'Interstellar'

In [57]:
# Créer le dossier models s'il n'existe pas
models_dir = DATA_DIR 

# Sauvegarder la liste des titres
liste_titres = df['Titre'].tolist()
with open(models_dir / 'liste_titres.pkl', 'wb') as f:
    pickle.dump(liste_titres, f)
print(f"Sauvegardé: {models_dir / 'liste_titres.pkl'}")

# Sauvegarder les métadonnées du modèle
metadata = {
    'n_films': len(df),
    'n_features': features_vectorized.shape[1],
    'date_creation': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2',
    'embedding_dim': synopsis_embeddings.shape[1],
    'features': {
        'synopsis_embeddings': synopsis_embeddings.shape,
        'titre_tfidf': titre_tfidf.shape,
        'genres': genre_matrix.shape,
        'metadata': (note_scaled.shape, age_scaled.shape, pop_scaled.shape)
    },
    'poids': {
        'synopsis': 1.5,
        'titre': 1.0,
        'note': 0.75,
        'age': 0.5,
        'popularite': 1.0,
        'genres': 1.0
    },
    'approche': 'Embeddings sémantiques (synopsis) + TF-IDF (titres) + Métadonnées'
}

with open(models_dir / 'model_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print(f"Sauvegardé: {models_dir / 'model_metadata.pkl'}")

Sauvegardé: C:\Users\Melvin\Desktop\DATA\PORTFOLIO\Recommandation de films\data\liste_titres.pkl
Sauvegardé: C:\Users\Melvin\Desktop\DATA\PORTFOLIO\Recommandation de films\data\model_metadata.pkl
